# Parse Fouble list of Archive.org links for Kermode and Mayo podcast

Fouble has a list of podcasts from 2005 - 2022 here: 
- [Kermode and Mayo Film Review archive (2005-2022)](https://fourble.co.uk/podcast/kermodeandma)


These use archive.org links rather than an RSS feed.
Parse the html web page to extract the `text` and `href` attributes from the ordered list of podcasts


In [1]:
# Imports
from dotenv import find_dotenv
from dotenv import dotenv_values
config = dotenv_values(find_dotenv())
# NOTE: empty `.env` file was added beneath `src` directory. Ignored by gitignore rules.
import os
import sys
sys.path.append(os.path.dirname(find_dotenv()))
from notebooks.notebook_utils import DevData

# ----------------------------------------
import json
import re
from bs4 import BeautifulSoup
from urllib.parse import urlparse

In [2]:

fouble_dir = os.path.join(DevData().data_dir, "rss_feeds", "fourble_archive")
fouble_path = os.path.join(fouble_dir, "view-source_https___fourble.co.uk_podcast_kermodeandma.html")
out_data_path = os.path.join(fouble_dir, "fouble_feed.jsonl")
content = open(fouble_path, encoding = "ISO-8859-1").read()

In [3]:
data = BeautifulSoup(content, 'html.parser') 
# finding all li tags in ul and printing the text within it 
ordered_list = data.find("ol") 
items = list(ordered_list.find_all("li"))
domains = {"/download/kermodeandmayocollection",  "/download/kermodeandmayocollection2"}

def get_date(href):
  p = urlparse(href).path
  parts = os.path.split(p)
  if parts[0] in domains:
    s = parts[1][:6]
    Y = "20" + s[:2]
    M = s[2:4]
    D = s[4:]
    return f"{Y}/{M}/{D}"


def dictify(item):  
  href = item.a["href"]
  date = get_date(href)
  text = items[1].a.text
  return {"pub_date": date, "url": href, "title": text}
  
outdata = [dictify(item)  for item in items]


In [4]:
# write json-lines data to file
with open(out_data_path, "w") as fo:
  fo.write("\n".join([json.dumps(x) for x in outdata] ))